In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [ ]:
!pip install -U torchtext==0.8.0

In [ ]:
pip install transformers

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
np.random.seed(500)

In [ ]:
path = "drive/MyDrive/CIS519/FP/"

In [ ]:
df1 = pd.read_csv(path+"both_train.csv")
df2 = pd.read_csv(path+"both_test.csv")
df3 = pd.read_csv(path+"both_val.csv")
df4 = pd.read_csv(path+"schizoaffective.csv")
df5 = pd.read_csv(path+"AlAnon.csv")
df6 = pd.read_csv(path+"BorderlinePDisorder_10.csv")
df7 = pd.read_csv(path+"alcoholicsanonymous.csv")
df8 = pd.read_csv(path+"OCD.csv")

In [ ]:
df4.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df5.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df6.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df7.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df8.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)

In [ ]:
df4 = df4[df4["body"] != "[deleted]"]
df5 = df5[df5["body"] != "[deleted]"]
df6 = df6[df6["body"] != "[deleted]"]
df7 = df7[df7["body"] != "[deleted]"]
df8 = df8[df8["body"] != "[deleted]"]

In [ ]:
df4["class_id"] = [6]*len(df4)
df5["class_id"] = [7]*len(df5)
df6["class_id"] = [8]*len(df6)
df7["class_id"] = [7]*len(df7)
df8["class_id"] = [9]*len(df8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df4.dropna(inplace=True)
df5.dropna(inplace=True)
df6.dropna(inplace=True)
df7.dropna(inplace=True)
df8.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df4.rename(columns={"body":"post","id":"ID"}, inplace=True)
df5.rename(columns={"body":"post","id":"ID"}, inplace=True)
df6.rename(columns={"body":"post","id":"ID"}, inplace=True)
df7.rename(columns={"body":"post","id":"ID"}, inplace=True)
df8.rename(columns={"body":"post","id":"ID"}, inplace=True)

In [ ]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8]
corpus = pd.concat(frames, ignore_index=True)

In [ ]:
corpus

,ID,title,post,class_name,class_id
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5
...,...,...,...,...,...
23572,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9
23573,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9
23574,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9
23575,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9


In [ ]:
corpus["text"] = corpus.title + " " + corpus.post

In [ ]:
corpus

,ID,title,post,class_name,class_id,text
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5,a question about the third conditional. i was ...
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0,the epitome of my life i've recently requested...
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5,what are your favourites offbeat destinations ...
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5,synesthesia survey (what colour is each month ...
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5,"science ama series: i’m phil baran, and i’m he..."
...,...,...,...,...,...,...
23572,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9,Can you have ocd without having depression??? ...
23573,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9,"Hi! I'm an OCD therapist, and I'm new to reddi..."
23574,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9,Vent: my mom who I still live with tested posi...
23575,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9,Do “intrusive memories” count as intrusive tho...


SVM and NB

In [ ]:
#Drop additional columns
corpus.drop(columns=["title","post"], inplace=True)

#Drop blank rows
corpus['text'].dropna(inplace = True)

#Convert text to lowercase
corpus['text'] = [text.lower() for text in corpus['text']]

#Tokenization
corpus['text'] = [word_tokenize(text) for text in corpus['text']]

#WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for idx, text in enumerate(corpus['text']):
  finalWords = []
  word_net_lemmatizer = WordNetLemmatizer()
  for word, tag in pos_tag(text):
    if word not in stopwords.words('english') and word.isalpha():
      finalWords.append(word_net_lemmatizer.lemmatize(word, tag_map[tag[0]]))
  corpus.loc[idx, 'finalText'] = str(finalWords)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(corpus['finalText'], corpus['class_id'], test_size=0.3, shuffle=True)

In [ ]:
x_train

12266    ['thank', 'god', 'meal', 'shake', 'know', 'guy...
17900    ['mother', 'day', 'drinking', 'eat', 'day', 'm...
14585    ['anyone', 'reckon', 'med', 'life', 'want', 't...
1927     ['hear', 'people', 'laugh', 'always', 'assume'...
22790    ['intrusive', 'thought', 'ruminate', 'old', 'f...
                               ...                        
19389    ['celebrate', 'one', 'month', 'milestone', 'tw...
3790     ['friend', 'drift', 'apart', 'super', 'depress...
8465     ['third', 'day', 'partial', 'hospitalization',...
12097    ['irritate', 'people', 'say', 'suicide', 'self...
17335    ['wo', 'suffer', 'silence', 'word', 'never', '...
Name: finalText, Length: 16503, dtype: object

In [ ]:
y_train.value_counts()

7     3513
3     2106
0     2063
2     2018
1     2010
4     1749
5     1722
10     567
8      449
6      306
Name: class_id, dtype: int64

In [ ]:
TFIDF_vect = TfidfVectorizer(max_features=5000)
x_train_tfidf = TFIDF_vect.fit_transform(x_train)
x_test_tfidf = TFIDF_vect.transform(x_test)

In [ ]:
TFIDF_vect.vocabulary_

{'thank': 4446,
 'god': 1954,
 'meal': 2756,
 'shake': 3954,
 'know': 2506,
 'guy': 2024,
 'bad': 373,
 'bout': 548,
 'attack': 323,
 'appetite': 252,
 'put': 3517,
 'food': 1796,
 'mouth': 2891,
 'like': 2604,
 'chew': 737,
 'tissue': 4508,
 'paper': 3168,
 'barely': 394,
 'manage': 2709,
 'essential': 1557,
 'energy': 1508,
 'use': 4730,
 'protein': 3471,
 'replacement': 3686,
 'sure': 4332,
 'available': 352,
 'usa': 4728,
 'make': 2702,
 'get': 1935,
 'etc': 1565,
 'without': 4930,
 'pressure': 3397,
 'eat': 1432,
 'think': 4466,
 'might': 2815,
 'useful': 4732,
 'share': 3964,
 'mother': 2882,
 'day': 1124,
 'drinking': 1389,
 'episode': 1540,
 'drink': 1387,
 'every': 1580,
 'one': 3079,
 'water': 4851,
 'need': 2948,
 'something': 4110,
 'though': 4472,
 'anyone': 231,
 'med': 2767,
 'life': 2597,
 'want': 4834,
 'take': 4378,
 'particularly': 3188,
 'kinda': 2492,
 'anti': 222,
 'max': 2749,
 'dose': 1356,
 'year': 4978,
 'worry': 4948,
 'end': 1500,
 'least': 2569,
 'could': 1

In [ ]:
print(x_train_tfidf)

  (0, 3964)	0.10244902250571493
  (0, 4732)	0.17381003882449517
  (0, 2815)	0.10505206933275674
  (0, 4466)	0.06060047475466629
  (0, 1432)	0.11761458336020535
  (0, 3397)	0.15594889162364595
  (0, 4930)	0.09331159525696829
  (0, 1565)	0.11023993747737701
  (0, 1935)	0.0472471311370443
  (0, 2702)	0.05706568152896919
  (0, 4728)	0.20283023624729718
  (0, 352)	0.15870845416099952
  (0, 4332)	0.19112794107355421
  (0, 3686)	0.21947461985880118
  (0, 3471)	0.21118498098803865
  (0, 4730)	0.08469797535394707
  (0, 1508)	0.13616006595324834
  (0, 1557)	0.198858676168157
  (0, 2709)	0.12566801302340092
  (0, 394)	0.1382305270508495
  (0, 3168)	0.15274000204283886
  (0, 4508)	0.22405944949429943
  (0, 737)	0.2164473460655489
  (0, 2604)	0.09794935420651178
  (0, 2891)	0.1667190942568488
  :	:
  (16502, 4014)	0.30643094717233604
  (16502, 4300)	0.2162374363009731
  (16502, 3247)	0.14072862800927707
  (16502, 1555)	0.10502595974650954
  (16502, 436)	0.11987404520257858
  (16502, 170)	0.06934871

In [ ]:
#Naive Bayes Classifier Algorithm
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train_tfidf,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(x_test_tfidf)

In [ ]:
#SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train_tfidf, y_train)
predictions_SVM = SVM.predict(x_test_tfidf)

In [ ]:
print("Naive Bayes Accuracy Score = ",accuracy_score(predictions_NB, y_test)*100)
print("Naive Bayes F1 Score = ",f1_score(predictions_NB, y_test, average='weighted')*100)

Naive Bayes Accuracy Score =  72.20808594854397
Naive Bayes F1 Score =  74.69981656913087


In [ ]:
print("SVM Accuracy Score = ",accuracy_score(predictions_SVM, y_test)*100)
print("SVM F1 Score = ",f1_score(predictions_SVM, y_test, average='weighted')*100)

SVM Accuracy Score =  79.9123551031948
SVM F1 Score =  80.129733655511
